4th order runge kutta with adapted step size

- small time step to improve accuracy

- integration more efficient (partition)

## a simple coupled ODE

d^2y/dx^2 = -y

for all x the second derivative of y is = -y (sin or cos curve)

- specify boundary conditions to determine which

- y(0) = 0 and dy/dx (x = 0) = 1 --> sin(x)

rewrte as coupled ODEs to solve numerically (slide 8)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#define coupled derivatives to integrate
def dydx(x,y):
    #y is a 2D array
    
    #equation is d^2y/dx^2 = -y
    #so: dydx = z, dz/dx = -y
    
    #set y = y[0], z = y[1]
    
    #declare array
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -1*y[0]
    
    return y_derivs

#can't evolve one without evolving the other, dependent variables

In [4]:
#define 4th order RK method
def rk_mv_core(dydx,xi,yi,nv,h):
    
    #nv = number of variables
    # h = width
    
    #declare k arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at half step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #find k1 values
    y_derivs = dydx(xi,yi)                  #array of y derivatives
    k1[:] = h*y_derivs[:]                   #taking diff euler steps for derivs
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    k4[:] = h*y_derivs[:]
    
    #advance y by step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.  #this is an array 
    
    return yipo

before, we took a single step

now we take two different versions of the same equation for the step

can be used as a check for the previous technique

the difference should be within tolerance to be valid (if the steps are too big and outside of tolerance then they need to be smaller bebeh steps)

In [6]:
#define adaptive step size for RK4
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY    = 0.9
    H_NEW_FAC = 2.0
    
    #set max number of iterations
    imax = 10000
    
    #set iteration variable, num of iterations taken
    i = 0
    
    #create an error (array)
    Delta = np.full(nv,2*tol)                                #twice the tol, if it exceeds tol
                                                             #steps need to be smoler
    
    #remember step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):                            #while loop
        #estimate error by taking one step of size h vs two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,0.5*h_step)
        
        #compute error
        Delta = np.fabs(y_2 - y_1)
        
        #if the error is too large
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)   #decreases h step size
            
        #check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
    
    #leave while loop, to try bigger steps
    h_new = np.fmin(h_step * (Delta.amx()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, the new step, and the step actually taken
    return y_2, h_new, h_step

In [ ]:
#wrapper function
def rk4_mv(dydx,a,b,y_a,tol):
    #dydx = deriv wrt x
    #a = lower bound
    #b = upper bound
    #y_a = boundary conditions (0,1)
    #tol = tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #initial step size (smallllll)
    h = 1.0e-4 * (b-a)
    
    #max number of iterations
    imax = 10000
    
    #set iteration variable
    i = 0
    
    #set the number of coupled ODEs to the size of y_a
    nv = len(y-a)
    
    #set initial conditions
    x = np.sull(1,a)
    y = np.full((1,nv),y_a)                   #2 dimensional array
    
    #set flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):